In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
from keras import layers
import numpy as np

batch = [np.random.random((224, 224, 1)) for _ in range(100)]
batch = np.array(batch)
batch.shape

E0000 00:00:1739464577.350611   63153 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739464577.353017   63153 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(100, 224, 224, 1)

In [3]:
from models.common import conv_block

b1 = conv_block(32, 3, 2, padding='valid', activation='relu')
b2 = conv_block(32, 3, 1, padding='valid', activation='relu')
b3 = conv_block(64, 3, 1, activation='relu')
z = b1(batch)
print(z.shape)
z = b2(z)
print(z.shape)
z = b3(z)
print(z.shape)

(100, 111, 111, 32)
(100, 109, 109, 32)
(100, 109, 109, 64)


In [4]:
b4 = layers.MaxPool2D(3, 2)
z = b4(z)
z.shape

TensorShape([100, 54, 54, 64])

In [5]:
b5 = conv_block(80, 1, padding='valid', activation='relu')
b6 = conv_block(192, 3, padding='valid', activation='relu')
b7 = conv_block(288, 3, 2, padding='valid', activation='relu')
z = b5(z)
print(z.shape)
z = b6(z)
print(z.shape)
z = b7(z)
print(z.shape)

(100, 54, 54, 80)
(100, 52, 52, 192)
(100, 25, 25, 288)


In [6]:
from models.inception_v3 import InceptionV3A

ia = InceptionV3A()
z = ia(z)
z.shape

TensorShape([100, 25, 25, 288])

In [7]:
ia2 = InceptionV3A()
z = ia2(z)
z.shape

TensorShape([100, 25, 25, 288])

In [8]:
from keras import layers, models

b3x3_1 = conv_block(384, 3, 2, padding="valid", activation='relu')
b3x3_2 = models.Sequential([
    conv_block(64, 1, activation='relu'),
    conv_block(96, 3, activation='relu'),
    conv_block(96, 3, 2, padding='valid', activation='relu'),
])
pool = layers.MaxPool2D(3, 2)
z = layers.Concatenate()([b3x3_1(z), b3x3_2(z), pool(z)])
z.shape

TensorShape([100, 12, 12, 768])

In [9]:
from models.inception_v3 import InceptionV3B

for _ in range(5):
    z = InceptionV3B()(z)
z.shape

TensorShape([100, 12, 12, 768])

In [10]:
from keras import layers, models

b3x3_1 = models.Sequential([
    conv_block(192, 1, activation='relu'),
    conv_block(320, 3, 2, padding='valid', activation='relu'),
])
b3x3_2 = models.Sequential([
    conv_block(192, 1, activation='relu'),
    conv_block(192, (1, 7), activation='relu'),
    conv_block(192, (7, 1), activation='relu'),
    conv_block(192, 3, 2, padding='valid', activation='relu'),
])
pool = layers.MaxPool2D(3, 2)
z = layers.Concatenate()([b3x3_1(z), b3x3_2(z), pool(z)])
z.shape

TensorShape([100, 5, 5, 1280])

In [11]:
from models.inception_v3 import InceptionV3C

for _ in range(2):
    z = InceptionV3C()(z)
z.shape

TensorShape([100, 5, 5, 2048])

In [12]:
z = layers.AvgPool2D(5)(z)
z.shape

TensorShape([100, 1, 1, 2048])

In [13]:
z = layers.Dropout(0.2)(z)
z.shape

TensorShape([100, 1, 1, 2048])

In [14]:
z = layers.Flatten()(z)
z.shape

TensorShape([100, 2048])

In [15]:
z = layers.Dense(1024, activation='linear')(z)
z.shape

TensorShape([100, 1024])

In [16]:
z = layers.Dropout(0.5)(z)
z.shape

TensorShape([100, 1024])

In [17]:
z = layers.Dense(2, activation='softmax')(z)
z.shape

TensorShape([100, 2])